In [1]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding

Using TensorFlow backend.


In [2]:
data = pd.read_csv('Dataset/hotel_data.csv')

In [3]:
data.head(5)

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,...,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id
0,Hardasji Ki Magri,Udaipur,India,2016-06-21,{{facility}},|Zion Home Stay is located in a city that sets...,1 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,78ddf880bd7937d384ff278cc5b39d6e
1,Near Nai Gaon,Udaipur,India,2016-06-21,{{facility}},| Araliayas Resorts is a 3 star hotel located ...,3 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,9f9f9cbb2f7df8089b63d5cdeb257944
2,Near Bagore Ki Haveli,Udaipur,India,2016-06-21,{{facility}},|A 2 star property is located at 24 km from Ma...,2 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,NaN,makemytrip,NaN,NaN,b314bb7fa8bfb1ed306f517be21d729e
3,Dabok,Udaipur,India,2016-06-21,Airport Transfer|Car rental|Conference Hall|Cu...,|SNP House Airport Hotel And Restaurant is loa...,1 star,//imghtlak.mmtcdn.com/images/hotels/2014071815...,NaN,no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,`standard,/5,NaN,makemytrip,NaN,NaN,e6f5bb3c2d76a78d978b9ceb0e31ec56
4,East Udaipur,Udaipur,India,2016-06-21,{{facility}},| Hotel Pichola Haveli is situated in the beau...,2 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,3.7,makemytrip,NaN,NaN,63072c301427b6ca450d31eea127bcf0


In [4]:
data.city.value_counts()

NewDelhiAndNCR        1163
Goa                   1122
Mumbai                 543
Jaipur                 534
Bangalore              512
                      ... 
Gajraula                 1
Chamba Uttaranchal       1
Krishnanagar             1
Nagarholae               1
Bijapur                  1
Name: city, Length: 770, dtype: int64

In [5]:
array = ['Mumbai']

In [6]:
data = data.loc[data['city'].isin(array)]

In [7]:
data.head(5)

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,...,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id
294,Charai,Mumbai,India,2016-08-28,Doctor on Call|Front desk|Laundry Service|Park...,"Nestled in Mumbai, a city with strong historic...",3,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Ac Superior Room|A/c Standard Rooms Double Occ...,NaN,NaN,makemytrip,Maharashtra,NaN,d78fae90ef2e1b5c2dfd547c61763a25
309,Andheri (East),Mumbai,India,2016-08-28,Air Conditioned|Airport Transfer|Conference Ha...,3 km from Chhatrapati Shivaji International Ai...,2,NaN,Bathroom Toiletries|Daily Newspaper|Kitchenett...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Executive|Deluxe,NaN,NaN,makemytrip,Maharasta,Location:3.2/5 | Hospitality:3.1/5 | Facilitie...,030865f741982beb373efddecdc6d6c3
321,Khar,Mumbai,India,2016-08-28,Airport/Rlwy Stn Transfer|Bar|Conference Hall|...,Location Hotel Royal Garden is situated on Juh...,3,NaN,Electronic Safe|Bathroom Toiletries|Daily News...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Superior Executive,NaN,NaN,makemytrip,Maharashtra,Location:4.5/5 | Hospitality:3.4/5 | Facilitie...,a1ced509350038775a7700ec67796bc2
334,Andheri (East),Mumbai,India,2016-08-28,24 Hour Check in-Icon|24 hour reception|24 hou...,City Guest House is a beautiful property locat...,2,NaN,Bathroom Toiletries|Hot/cold Water|Attached Ba...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Standard Room|Deluxe Room|Triple Deluxe Room|S...,NaN,NaN,makemytrip,Maharashtra,Location:4.3/5 | Hospitality:3.8/5 | Facilitie...,f2820ae7707668ed6906bb227921f720
1238,Andheri (East),Mumbai,NaN,2016-08-22,24 Hour Check in-Icon|24 hour reception|24 hou...,Sai Residency Hotel is situated in the City of...,2,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,no,...,Hotel,2016-08-22 22:10:53 +0000,2016-08-22 22:10:53 +0000,Deluxe Dbl Air Cooled,NaN,NaN,makemytrip,MAHARASHTRA,NaN,b4af24952027ffbcd85a91bb6fe23f5d


In [8]:
data = data.hotel_overview
data = data.dropna()

In [9]:
stop = set(stopwords.words('english'))
def stopwords_removal(data_point):
    data = [x for x in data_point.split() if x not in stop]
    return data

In [10]:
def clean_data(data):
    cleaned_data = []
    all_unique_words_in_each_description = []
    for entry in data:
        entry = re.sub(pattern='[^a-zA-Z]',repl=' ',string = entry)
        entry = re.sub(r'\b\w{0,1}\b', repl=' ',string = entry)
        entry = entry.lower()
        entry = stopwords_removal(entry)
        cleaned_data.append(entry)
        unique = list(set(entry))
        all_unique_words_in_each_description.extend(unique)
    return cleaned_data, all_unique_words_in_each_description

In [11]:
def unique_words(data):
    unique_words = set(all_unique_words_in_each_description)
    return unique_words, len(unique_words)

In [12]:
cleaned_data, all_unique_words_in_each_description = clean_data(data)
unique_words, length_of_unique_words = unique_words(all_unique_words_in_each_description)

In [13]:
cleaned_data[0]

['nestled',
 'mumbai',
 'city',
 'strong',
 'historical',
 'links',
 'wonderful',
 'british',
 'architecture',
 'museums',
 'beaches',
 'places',
 'worship',
 'true',
 'galaxy',
 'stars',
 'bollywood',
 'reigns',
 'supreme',
 'hotel',
 'divya',
 'international',
 'delightful',
 'leisure',
 'absolute',
 'blend',
 'service',
 'charm',
 'efficiency',
 'hotel',
 'offers',
 'facilities',
 'like',
 'front',
 'desk',
 'parking',
 'laundry',
 'doctor',
 'call',
 'many',
 'aims',
 'extend',
 'best',
 'possible',
 'hospitality',
 'experience',
 'revered',
 'customers',
 'hotel',
 'located',
 'distance',
 'workshop',
 'bus',
 'stop',
 'km',
 'chhatrapati',
 'shivaji',
 'international',
 'airport',
 'km',
 'chhatrapati',
 'shivaji',
 'terminus',
 'guests',
 'head',
 'prominent',
 'tourist',
 'attractions',
 'like',
 'sanjay',
 'gandhi',
 'national',
 'park',
 'kidzania',
 'mumbai',
 'haji',
 'ali',
 'mosque',
 'iskon',
 'temple',
 'shree',
 'siddhivinayak',
 'temple',
 'many',
 'shoppers',
 'fun',

In [14]:
length_of_unique_words

3395

In [15]:
def build_indices(unique_words):
    word_to_idx = {}
    idx_to_word = {}
    for i, word in enumerate(unique_words):
        word_to_idx[word] = i
        idx_to_word[i] = word
    return word_to_idx, idx_to_word

In [16]:
word_to_idx, idx_to_word = build_indices(unique_words)

In [17]:
def prepare_corpus(corpus, word_to_idx):
    
    sequences = []
    for line in corpus:
        tokens = line
        for i in range(1, len(tokens)):
            i_gram_sequence = tokens[:i+1]
            i_gram_sequence_ids = []
            
            for j, token in enumerate(i_gram_sequence):
                i_gram_sequence_ids.append(word_to_idx[token])
                
            sequences.append(i_gram_sequence_ids)
    
    return sequences

In [18]:
sequences = prepare_corpus(cleaned_data, word_to_idx)
max_sequence_len = max([len(x) for x in sequences])

In [59]:
print(sequences[0])
print(sequences[1])

[1647, 867]
[1647, 867, 1452]


In [61]:
print(idx_to_word[1647])
print(idx_to_word[867])
print(idx_to_word[1452])

nestled
mumbai
city


In [19]:
len(sequences)

51836

In [20]:
max_sequence_len

308

In [21]:
def build_input_data(sequences, max_sequence_len, length_of_unique_words):
    sequences = np.array(pad_sequences(sequences, maxlen = max_sequence_len, padding = 'pre'))
    X = sequences[:,:-1]
    y = sequences[:,-1]
    y = np_utils.to_categorical(y, length_of_unique_words)
    return X, y

In [22]:
X, y = build_input_data(sequences, max_sequence_len, length_of_unique_words)

In [23]:
def create_model(max_sequence_len, length_of_unique_words):
    model = Sequential()
    model.add(Embedding(length_of_unique_words, 10, input_length=max_sequence_len - 1))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(length_of_unique_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [24]:
model = create_model(max_sequence_len, length_of_unique_words)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 307, 10)           33950     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               71168     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3395)              437955    
Total params: 543,073
Trainable params: 543,073
Non-trainable params: 0
_________________________________________________________________


In [25]:
len(X)

51836

In [ ]:
model.fit(X, y, batch_size = 512, epochs=100)

Epoch 1/100
51836/51836 [==============================] - 157s 3ms/step - loss: 6.9315
Epoch 2/100
51836/51836 [==============================] - 152s 3ms/step - loss: 6.5816
Epoch 3/100
51836/51836 [==============================] - 156s 3ms/step - loss: 6.5273
Epoch 4/100
51836/51836 [==============================] - 159s 3ms/step - loss: 6.4325
Epoch 5/100
51836/51836 [==============================] - 157s 3ms/step - loss: 6.2997
Epoch 6/100
51836/51836 [==============================] - 157s 3ms/step - loss: 6.2009
Epoch 7/100
51836/51836 [==============================] - 165s 3ms/step - loss: 6.2528
Epoch 8/100
51836/51836 [==============================] - 162s 3ms/step - loss: 6.0584
Epoch 9/100
51836/51836 [==============================] - 155s 3ms/step - loss: 5.9684
Epoch 10/100
51836/51836 [==============================] - 155s 3ms/step - loss: 5.8849
Epoch 11/100
51836/51836 [==============================] - 155s 3ms/step - loss: 5.7997
Epoch 12/100
51836/51836 [====

In [31]:
def generate_text(seed_text, next_words, model, max_seq_len):
    for _ in range(next_words):
        cleaned_data = clean_data([seed_text])
        sequences= prepare_corpus(cleaned_data[0], word_to_idx)
        sequences = pad_sequences([sequences[-1]], maxlen=max_seq_len-1, padding='pre')
        predicted = model.predict_classes(sequences, verbose=0)
        output_word = ''
        output_word = idx_to_word[predicted[0]]            
        seed_text = seed_text + " " + output_word
        
    return seed_text.title()

In [33]:
print(generate_text("in Mumbai there we need", 30, model, max_sequence_len))

In Mumbai There We Need Located Mumbai City Mumbai Charismatic Electrifying Open Hearted Mumbai Bombay City Dreamers Stalwarts Common Man Guests Visit Majestic Places Like Gateway India Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International


In [44]:
print(generate_text("Best Hotel Mumbai", 30, model, max_sequence_len))

Best Hotel Mumbai Situated Mumbai City Diverse Cultures Distinct Lifestyles Surprising Mix Modernity Urbanism Along Age Old Traditions Customs Everyone Visits Falls Love City Beauty Chaos Intoxicating Charm Hotel Offers Facilities Like Front


In [45]:
print(generate_text("The beauty of the city", 30, model, max_sequence_len))

The Beauty Of The City World Pilgrimage Employment Opportunities Park Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Vile Parle Railway Station Km Kamgar Hospital Bus Stand Prominent Tourist Spots Like Tikuji


In [57]:
model_structure = model.to_json()
with open("Output Files/text_generation_using_LSTM.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights("Output Files/text_generation_using_LSTM.h5")